# ConnorsReversal Strategy Batch Optimization

这个notebook用于对多个交易对进行ConnorsReversal策略的批量优化。

In [1]:
import backtrader as bt
import pandas as pd
from datetime import datetime
import numpy as np
import os
from app.plugins.connors_reversal_plugin import Plugin

ModuleNotFoundError: No module named 'app'

In [2]:
# 配置示例
CONFIG = {
    'data_path': r'\\znas\Main\futures',
    'start_date': '2024-01-01',
    'end_date': '2024-02-01',
    'reports_path': 'reports',
    'symbols_selection': {
        'mode': 'specific',
        'specific_symbols': ['BTC', 'ETH']
    },
    'score_weights': {
        'ret_weight': 0.6,
        'sqn_weight': 0.4,
        'sharpe_weight': 0.2,
        'win_rate_weight': 0.15,
        'dd_weight': 0.1,
        'min_trades': 50
    }
}

In [3]:
class HeuristicBatchTester:
    def __init__(self, config):
        self.config = config
        self.plugin = Plugin()
        self.data_cache = {}
        
    def run_batch_test(self):
        """运行批量回测"""
        # 确保报告目录存在
        os.makedirs(self.config['reports_path'], exist_ok=True)
        
        # 获取要处理的交易对列表
        symbols = self._get_symbols()
        print(f"将回测 {len(symbols)} 个交易对")
        
        # 遍历每个交易对
        for symbol in symbols:
            self._test_single_symbol(symbol)
            
    def _test_single_symbol(self, symbol):
        """测试单个交易对"""
        print(f"\n正在处理交易对: {symbol}")
        print("=" * 50)
        
        # 遍历每个时间周期
        for timeframe in ['30min', '15min', '5min', '1min']:
            try:
                # 准备配置
                test_config = {
                    'symbol': symbol,
                    'target_timeframe': timeframe,
                    'start_date': self.config['start_date'],
                    'end_date': self.config['end_date'],
                    'data_path': self.config['data_path'],
                    'score_weights': self.config['score_weights']
                }
                
                # 运行回测
                results = self._run_single_test(test_config)
                
                # 保存结果
                self._save_results(symbol, timeframe, results)
                
            except Exception as e:
                print(f"错误: 处理 {symbol}-{timeframe} 时发生异常: {str(e)}")
                continue
                
    def _run_single_test(self, config):
        """运行单个回测"""
        # 使用插件的评估方法
        individual = (
            self.plugin.params['lowest_point_bars'],
            self.plugin.params['rsi_length'],
            self.plugin.params['sell_barrier'],
            self.plugin.params['dca_parts']
        )
        
        # 运行评估
        score = self.plugin.evaluate_candidate(
            individual=individual,
            base_data=None,  # 这里不需要预测数据
            hourly_predictions=None,
            daily_predictions=None,
            config=config
        )
        
        return score

    def _get_symbols(self):
        """获取要处理的交易对列表"""
        if self.config['symbols_selection']['mode'] == 'specific':
            return self.config['symbols_selection']['specific_symbols']
        else:
            # 实现获取所有交易对的逻辑
            return self._get_all_symbols()
            
    def _save_results(self, symbol, timeframe, results):
        """保存回测结果"""
        results_file = os.path.join(
            self.config['reports_path'],
            f"backtest_results_{symbol}_{timeframe}.csv"
        )
        
        # 将结果保存为DataFrame
        df = pd.DataFrame([results])
        df.to_csv(results_file, index=False)


In [6]:
# 使用示例
def main():
    batch_tester = HeuristicBatchTester(CONFIG)
    batch_tester.run_batch_test()

In [ ]:


if __name__ == '__main__':
    main()

将优化 4 个交易对

开始逐个处理交易对...

正在处理交易对: BTCUSDT

加载 BTCUSDT 的数据...
加载 30min 时间周期...
加载 15min 时间周期...
加载 5min 时间周期...
加载 1min 时间周期...

成功加载 BTCUSDT 的 4 个时间周期数据

开始优化 BTCUSDT-30min
开始优化 BTCUSDT 在 30min 上的策略...


[I 2025-02-23 16:36:11,291] Using an existing study with name 'BTCUSDT_30min' instead of creating a new one.


  0%|          | 0/10 [00:00<?, ?it/s]

2025-02-01 策略启动 - Warmup: 110, MaxWindow: 50
试验失败: {'params': {'lowest_point_bars': 45, 'rsi_length': 2, 'sell_barrier': 70, 'dca_parts': 4}, 'error': 'float division by zero', 'timestamp': Timestamp('2025-02-23 16:38:23.989707')}
[I 2025-02-23 16:38:24,044] Trial 10 finished with value: -1000.0 and parameters: {'lowest_point_bars': 45, 'rsi_length': 2, 'sell_barrier': 70, 'dca_parts': 4}. Best is trial 0 with value: -1000.0.


[I 2025-02-23 16:39:10,560] Using an existing study with name 'BTCUSDT_15min' instead of creating a new one.


[W 2025-02-23 16:39:10,484] Trial 11 failed with parameters: {'lowest_point_bars': 45, 'rsi_length': 37, 'sell_barrier': 75, 'dca_parts': 8} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\x7498\anaconda3\envs\backtesting\lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\x7498\AppData\Local\Temp\ipykernel_20804\312880326.py", line 55, in objective
    results = cerebro.run()
  File "c:\Users\x7498\anaconda3\envs\backtesting\lib\site-packages\backtrader\cerebro.py", line 1132, in run
    runstrat = self.runstrategies(iterstrat)
  File "c:\Users\x7498\anaconda3\envs\backtesting\lib\site-packages\backtrader\cerebro.py", line 1217, in runstrategies
    data.preload()
  File "c:\Users\x7498\anaconda3\envs\backtesting\lib\site-packages\backtrader\feed.py", line 439, in preload
    while self.load():
  File "c:\Users\x7498\anaconda3\envs\backtesting\lib\site-packages

  0%|          | 0/10 [00:00<?, ?it/s]